# Clustering of Wikipedia Names

In [ ]:
import re
import csv
from unidecode import unidecode
from urllib.parse import unquote
from fog.clustering import key_collision
from fog.key import fingerprint, rusalka
from fog.tokenizers import fingerprint_tokenizer
from fog.phonetics.utils import squeeze
from tqdm import tqdm_notebook

## Constants

In [ ]:
LANGS = {
    'english': 'en',
    'french': 'fr',
    'spanish': 'es',
    'portuguese': 'pt',
    'german': 'de',
    'italian': 'it',
    'swedish': 'sv'
}

## Helpers

In [ ]:
def is_cluster_relevant(cluster, results_index=None):
    """
    Function returning whether a cluster is relevant and the set of found ids.
    
    A cluster will be deemed relevant if:
        1) It contains names from more than one single id.
        2) If the range between birth/death does not exceed a given threshold.

    """
    ids = set()
    exact_birth_dates = set()
    
    for person in cluster:
        ids.add(person['id'])
        
        exact_birth_date = person['exact_birth']
        
        if exact_birth_date is not None:
            exact_birth_dates.add(exact_birth_date)

    if len(ids) == 1:
        return False, ids
    
    if len(exact_birth_dates) > 1:
        return False, ids
    
    if results_index is not None:
        result = results_index.get(tuple(sorted(ids)))

        if result is not None:
            return False, ids
    
    return True, ids

In [ ]:
def score_cluster(cluster):
    if len(set(p['birth'] for p in cluster if p['birth'] is not None)) == 1 and \
       len(set(p['death'] for p in cluster if p['death'] is not None)) == 1:
        return 'High'
    
    if any(p for p in cluster if p['exact_birth'] is None):
        return 'Low'
    
    return 'High'

In [ ]:
def print_cluster(cluster, ids):
    print('Found cluster containing %i ids and %i persons:' % (len(ids), len(cluster)))
    
    for person in cluster:
        print('  %s (%i) (%s - %s) (%s)' % (person['name'], person['id'], person['birth'], person['death'], person['lang']))
        
    print()

In [ ]:
def print_cluster_html(cluster, ids):
        
    confidence = score_cluster(cluster)
    
    print('<div>')
    print('  <p>')
    print('    (%s confidence) Found cluster containing %i ids and %i persons:' % (confidence, len(ids), len(cluster)))
    print('  </p>')
    
    print('  <ul>')
    for person in cluster:
        link = 'https://%s.wikipedia.org/wiki/%s' % (person['lang'], person['name'])
        print('    <li>%s (%i) (%s - %s) (%s) (%s) <u>(<a href="%s" target="_blank">link</a>)</u></li>' % (person['name'], person['id'], person['birth'], person['death'], person['exact_birth'], person['lang'], link))
    print('  </ul>')
        
    print('</div>')

## Processing data

In [ ]:
INPUT = './clustering.csv'
BIRTH_DATE_INPUT = './birthdates.csv'
OUTPUT = './found.csv'
BIRTH_DATE_INDEX = {}
PERSONS = []

with open(BIRTH_DATE_INPUT, 'r') as f:
    reader = csv.DictReader(f)
    
    for line in reader:
        BIRTH_DATE_INDEX[(line['lang'], line['name'])] = line['birth_date']

with open(INPUT, 'r') as f:
    reader = csv.DictReader(f)
    
    for line in tqdm_notebook(reader):
        for lang in LANGS:
            name = line['%s_link' % lang]
            
            if not name:
                continue
                
            person = {
                'id': int(line['id']),
                'lang': LANGS[lang],
                'name': name,
                'birth': int(line['birth_min']) if line['birth_min'] else None,
                'death': int(line['death_min']) if line['death_min'] else None,
                'exact_birth': BIRTH_DATE_INDEX.get((LANGS[lang], name)) or None
            }
            
            PERSONS.append(person)

In [ ]:
RESULTS_SO_FAR = './clustering_results.csv'
RESULTS_INDEX = {}
with open(RESULTS_SO_FAR, 'r') as f:
    reader = csv.DictReader(f)
    
    for line in reader:
        ids = tuple(sorted(int(i) for i in line['ids'].split('|')))
        
        RESULTS_INDEX[ids] = line['result']

## Connected components

In [ ]:
def grouper(i):
    x = PERSONS[i]
    
    if not x['birth']:
        return None
    
    f = x['name'].lower()
    
    return (x['birth'], x['death'], f)

clusters = key_collision(range(len(PERSONS)), key=grouper)

of = open(OUTPUT, 'w')
writer = csv.DictWriter(of, fieldnames=['lang', 'name'])
writer.writeheader()

RELEVANT_CLUSTERS = 0
for cluster in clusters:
    cluster = [PERSONS[i] for i in cluster]
    relevant, ids = is_cluster_relevant(cluster, RESULTS_INDEX)
    
    if not relevant or score_cluster(cluster) != 'High':
        continue
        
    RELEVANT_CLUSTERS += 1
        
    print_cluster_html(cluster, ids)
    # ids_str = '|'.join(set(str(p['id']) for p in cluster))
    # print('high_confidence_cc,%s,oui' % ids_str)
    for person in cluster:
        writer.writerow({'lang': person['lang'], 'name': person['name']})
    
print('Found %i relevant clusters' % RELEVANT_CLUSTERS)
of.close()

## Aggressive normalization

In [ ]:
STRIP_NONLETTERS_RE = re.compile(r'[^a-z0-9]')

def grouper(i):
    x = PERSONS[i]
    
    if not x['birth']:
        return None
    
    f = unquote(x['name'].lower())
    
    f = re.sub(STRIP_NONLETTERS_RE, '', f)

    return (x['birth'], x['death'], f)

clusters = key_collision(range(len(PERSONS)), key=grouper)

of = open(OUTPUT, 'w')
writer = csv.DictWriter(of, fieldnames=['lang', 'name'])
writer.writeheader()

RELEVANT_CLUSTERS = 0
for cluster in clusters:
    cluster = [PERSONS[i] for i in cluster]
    relevant, ids = is_cluster_relevant(cluster, RESULTS_INDEX)
    
    if not relevant:
        continue
        
    RELEVANT_CLUSTERS += 1
        
    print_cluster_html(cluster, ids)

    for person in cluster:
        writer.writerow({'lang': person['lang'], 'name': person['name']})
    
print('Found %i relevant clusters' % RELEVANT_CLUSTERS)
of.close()

## Accents

In [ ]:
STRIP_NONLETTERS_RE = re.compile(r'[^a-z0-9]')

def grouper(i):
    x = PERSONS[i]
    
    if not x['birth']:
        return None
    
    f = unquote(x['name'].lower())
    f = unidecode(f)
    
    f = re.sub(STRIP_NONLETTERS_RE, '', f)

    return (x['birth'], x['death'], f)

clusters = key_collision(range(len(PERSONS)), key=grouper)

of = open(OUTPUT, 'w')
writer = csv.DictWriter(of, fieldnames=['lang', 'name'])
writer.writeheader()

RELEVANT_CLUSTERS = 0
for cluster in clusters:
    cluster = [PERSONS[i] for i in cluster]
    relevant, ids = is_cluster_relevant(cluster, RESULTS_INDEX)
    
    if not relevant:
        continue
        
    RELEVANT_CLUSTERS += 1
        
    # print_cluster_html(cluster, ids)
    ids_str = '|'.join(set(str(p['id']) for p in cluster))
    print('accents,%s,oui' % ids_str)
    for person in cluster:
        writer.writerow({'lang': person['lang'], 'name': person['name']})
    
print('Found %i relevant clusters' % RELEVANT_CLUSTERS)
of.close()

## Fingerprinting

In [ ]:
def grouper(i):
    x = PERSONS[i]
    
    if not x['birth']:
        return None
    
    f = unquote(x['name'])
    f = f.replace('_', ' ').replace('-', ' ')
    f = fingerprint(f)

    return (x['birth'], x['death'], f)

clusters = key_collision(range(len(PERSONS)), key=grouper)

RELEVANT_CLUSTERS = 0
for cluster in clusters:
    cluster = [PERSONS[i] for i in cluster]
    relevant, ids = is_cluster_relevant(cluster, RESULTS_INDEX)
    
    if not relevant:
        continue
        
    RELEVANT_CLUSTERS += 1
        
    print_cluster_html(cluster, ids)
    # ids_str = '|'.join(set(str(p['id']) for p in cluster))
    # print('accents,%s,oui' % ids_str)
    
print('Found %i relevant clusters' % RELEVANT_CLUSTERS)

## Squeezed fingerprinting

In [ ]:
def grouper(i):
    x = PERSONS[i]
    
    if not x['birth']:
        return None
    
    f = unquote(x['name'])
    f = f.replace('_', ' ').replace('-', ' ')
    f = squeeze(f)
    f = fingerprint(f)

    return (x['birth'], x['death'], f)

of = open(OUTPUT, 'w')
writer = csv.DictWriter(of, fieldnames=['lang', 'name'])
writer.writeheader()

clusters = key_collision(range(len(PERSONS)), key=grouper)

RELEVANT_CLUSTERS = 0
for cluster in clusters:
    cluster = [PERSONS[i] for i in cluster]
    relevant, ids = is_cluster_relevant(cluster, RESULTS_INDEX)
    
    if not relevant:
        continue

    RELEVANT_CLUSTERS += 1
        
    print_cluster_html(cluster, ids)
    # ids_str = '|'.join(set(str(p['id']) for p in cluster))
    # print('accents,%s,oui' % ids_str)
    for person in cluster:
        writer.writerow({'lang': person['lang'], 'name': person['name']})
    
print('Found %i relevant clusters' % RELEVANT_CLUSTERS)
of.close()

## Rusalka

In [ ]:
NUM_RE = re.compile('\d')

def grouper(i):
    x = PERSONS[i]
    
    if not x['birth']:
        return None
    
    f = unquote(x['name'])
    f = f.replace('_', ' ').replace('-', ' ')
    f = squeeze(f)
    f = fingerprint_tokenizer(f)
    
    f = [i for i in f if len(i) > 2 and not re.match(NUM_RE, i)]
    
    if len(f) == 0:
        return None
    
    f = ' '.join(rusalka(i) for i in f)

    return (x['birth'], x['death'], f)

of = open(OUTPUT, 'w')
writer = csv.DictWriter(of, fieldnames=['lang', 'name'])
writer.writeheader()

clusters = key_collision(range(len(PERSONS)), key=grouper)

RELEVANT_CLUSTERS = 0
for cluster in clusters:
    cluster = [PERSONS[i] for i in cluster]
    relevant, ids = is_cluster_relevant(cluster, RESULTS_INDEX)
    
    if not relevant:
        continue

    RELEVANT_CLUSTERS += 1
        
    print_cluster_html(cluster, ids)
    # ids_str = '|'.join(set(str(p['id']) for p in cluster))
    # print('accents,%s,oui' % ids_str)
    for person in cluster:
        writer.writerow({'lang': person['lang'], 'name': person['name']})
    
print('Found %i relevant clusters' % RELEVANT_CLUSTERS)
of.close()